# todo

In [1]:
import netCDF4
import string

* **Done** Time = 480  can we make this lowercase? 'time'; if we do that in the definition up top it should carry it through all the variable dimensions, right?
* **not necessary, in fact my programm does not even allow it, it takes care of it it self** to the 'time' variable add an identifier for time zone :units="seconds since 1995-5-10 00:00:00 0:00" - so adding the final 0:00 after a space indicates 0 offset from UTC
* **Done** int time variable needs :epoch_time attribute - Ken says it's a number you need to calculate but trivial with Python utility and this will help users to calculate time easily without having to parse the other string/array
* **Done** multiple variables in the :ancillary_variables attribute should be separated by spaces and not commas so the commas should be removed
* **Done** variable 'diffuse_shortwave_irloss_flag' - "shortwave" is misspelled
* **Done** for variables 'wind_speed' and 'wind_direction' can we change the comments to "measured at top of tower"
* **Done ... i think** there are a couple of added variables in there that we're not sure what they're for  :type = "byte" and :Coordinates = "Time" - these pooped up in some of the variables and are not needed and potentially contradictory with the variable definition - can we take all of these out?
* **Done** add a //global_attribute :history = "this file was created on <time stamp> on the machine <name machine>" - this will help with troubleshooting down the road if needed

* **Yes** are you using netCDF 4 libraries?
* **Done** do you have the 'classic mode' switch on which makes it back-compatible? Ken recommends this

**disagreed, see section 2.4 ... http://cfconventions.org/Data/cf-conventions/cf-conventions-1.7/cf-conventions.html#dimensions** The "coordinates" attribute (attribute's name case matters) is intended to point to other coordinate variables that should be used to describe the data in space. There is an example of the use in section 5.3. Since you are just presenting a time series I suggest taking that attribute out.

**Gone** I can't find any description of an attribute called "type" in CF. I suggest taking it out.

**I prefere without it since my library is taking care of that it self and does not allow me to change it** CF section 4.4 has an example of using the "0:00" timezone offset if you want to understand that more. The Standard does say "Note: if the time zone is omitted the default is UTC, and if both time and time zone are omitted the default is 00:00:00 UTC." So you can apparently leave it off if you prefer.

Here is the description of "calendar" that makes me think what you have is correct: "For time coordinates that do cross the discontinuity the proleptic_gregorian calendar should be used instead."

# Questions

- i noticed that data is stored as float32 in the old data ... would prefer 64 ... problem?
- fillValue ... there is this "\_FillValue" parameter which don't know what it exatly mean ... is that actually the fillValue or does that mean that feature is not used ... why the underscore?!?
- not all variables have "standard_name"
- ancillary_variables is applied multiple time -> attribute can only be a string ... made a single string now

# imports 

# Using the cdl file

## TODO!! parse the cdl file for programmatic changes

In [640]:
pd.to_datetime('1995-05-10') - pd.to_timedelta(800064000, unit='s')

Timestamp('1970-01-01 00:00:00')

In [527]:
xr.version.version

'0.10.2'

In [528]:
netCDF4.__netcdf4libversion__

'4.5.0'

In [522]:
netCDF4.__version__

'1.3.1'

In [108]:
netCDF4.__hdf5libversion__

'1.10.1'

## generate the nc file

execute in terminal:
- cd ~/prog/SURFRAD/data/
- ncgen -o test.nc SURFRAD_QCrad_metadata.cdl

In [30]:
ds = xr.open_dataset('../data/test.nc', autoclose=True, 
                     chunks={'time': 480}
                    )

In [31]:
for var in ds.variables:
    enc = ds[var].encoding
    enc['chunksizes'] = enc['original_shape']
    enc['contiguous'] = False
    enc['complevel'] = 9
    enc['zlib'] = True

In [32]:
# ds.close()

# # %%debug -b /Users/htelg/anaconda3/lib/python3.6/site-packages/xarray/backends/netCDF4_.py:397
# ds.to_netcdf('../data/testII.nc')

# dst = xr.open_dataset('../data/testII.nc', autoclose=True)

# dst.close()



# dst = netCDF4.Dataset('../data/testII.nc')

# var = dst.variables['time']
# var.chunking()

# var.ncattrs()

# dst.close()

## load data

In [33]:
fname = '../data/bon_19950510.qdat'
data = pd.read_csv(fname,delim_whitespace=True)

# creating timestamp
datetimestr = data.Date.apply(lambda x: '{0:0>8}'.format(x)) + data.Time.apply(lambda x: '{0:0>4}'.format(x)) + 'UTC'#'+0000'

data.index = pd.to_datetime(datetimestr, format = "%Y%m%d%H%M%Z") 
data.index.name = 'time'

# removing obsolete columns
data = data.drop(['Date', 'Time'], axis=1)

In [34]:
# our missing data is np.nan
data[data == -9999.0] = np.nan
data[data == -9999.9] = np.nan

# pressure mbar to pa
data.Prs *= 100

In [35]:
# for col in data:
#     print('{}: {}, {}'.format(col,data[col].min(), data[col].max()))

In [36]:
# todo: this is only to match the number of values which is defined in the cdl file ... will try to adjust programmatically in the future
# data = data.iloc[:-1,:] 

## fill variables in dataset

In [37]:
columntransrules = {
 'solar_zenith_angle':                                                  'Z',
 'distance_from_sun':                                                   'AU',
 'best_estimate_shortwave_irradiance':                                  'BESW',
 'total_downwelling_shortwave_irradiance':                              'GSW',
 'diffuse_downwelling_shortwave_irradiance':                            'DIF',
 'direct_downwelling_shortwave_irradiance':                             'DIR',
 'upwelling_shortwave_irradiance':                                      'SWup',
 'downwelling_longwave_irradiance':                                     'LWdn',
 'upwelling_longwave_irradiance':                                       'LWup',
 'atmospheric_temperature':                                             'Ta',
 'relative_humidity':                                                   'RH',
 'surface_air_pressure':                                                'Prs',
 'downwelling_longwave_case_temperature':                               'LWdTc',
 'downwelling_longwave_dome_temperature':                               'LWdTd',
 'upwelling_longwave_case_temperature':                                 'LWuTc',
 'upwelling_longwave_dome_temperature':                                 'LWuTd',
 'QC_flag_01_thresholds_global_shortwave':                              'qc1',
 'QC_flag_02_thresholds_diffuse_shortwave':                             'qc2',
 'QC_flag_03_thresholds_direct_shortwave':                              'qc3',
 'QC_flag_04_thresholds_upwelling_shortwave':                           'qc4',
 'QC_flag_05_thresholds_downwelling_longwave':                          'qc5',
 'QC_flag_06_thresholds_upwelling_longwave':                            'qc6',
 'QC_flag_07_global_shortwave_over_sum':                                'qc7',
 'QC_flag_08_diffuse_over_global_shortwave':                            'qc8',
 'QC_flag_09_upwelling_shortwave_versus_sum':                           'qc9',
 'QC_flag_10_downwelling_longwave_to_temperature':                      'qc10',
 'QC_flag_11_upwelling_longwave_to_temperature':                        'qc11',
 'QC_flag_12_downwelling_longwave_to_upwelling_longwave':               'qc12',
 'QC_flag_13_downwelling_longwave_case_temperature_vs_air_temperature': 'qc13',
 'QC_flag_14_downwelling_longwave_dome_temperature_vs_air_temperature': 'qc14',
 'QC_flag_15_upwelling_longwave_case_temperature_vs_air_temperature':   'qc15',
 'QC_flag_16_upwelling_longwave_dome_temperature_vs_air_temperature':   'qc16',
 'QC_flag_17_downwelling_longwave_case_temperature_vs_dome_temperature':'qc17',
 'QC_flag_18_upwelling_longwave_case_temperature_vs_dome_temperature':  'qc18',
 'QC_flag_19_air_temperature':                                          'qc19',
 'global_shortwave_irloss_flag':                                        'gflg',
 'diffuse_shortwave_irloss_flag':                                        'dflg',
 'downwelling_shortwave_irradiance_assuming_clear_sky':                 'ClrSW',
 'ir_loss_correction_to_diffuse_shortwave':                             'DifCorr',
 'ir_loss_correction_to_global_shortwave':                              'GSWCorr',
 'downwelling_erythemal_uvb_irradiance':                                'UVB_down',
 'photosynthetically_active_radiation':                                 'PAR_dn',
 'wind_speed':                                                          'WindSp',
 'wind_direction':                                                      'WindDr',
}

location_values = { 'latitude': 40.05192,
                    'longitude': 271.62692,
                    'altitude': 230.}

In [38]:
ds.time.attrs

OrderedDict([('long_name', 'time in UTC (12:00am - 11:59pm)'),
             ('standard_name', 'time'),
             ('epoch_time', 800064000),
             ('_ChunkSize', 480)])

In [39]:
ds['time'].values = data.index

invalid = string.punctuation
for i in  {'_', '-', '.', '+', '@'}:
    invalid = invalid.replace(i,'')
invalid

e = 1
verbose = True
for varname, colname in columntransrules.items():
#     print(varname)
#     break
    ds[varname].values = data.loc[:,colname].values.astype(ds[varname].dtype)
    if 'flag_values' in ds[varname].attrs:
        flag_values_len = len(ds[varname].attrs['flag_values'])
        try:
            flag_meaning_len = len(ds[varname].attrs['flag_meanings'].split())
        except KeyError:
            print('-----')
            print('{}) {}: no flag_meanings found'.format(e,varname))
            e += 1
            if verbose:
                print('\tflag_values: {}'.format(ds[varname].attrs['flag_values']))
            continue
        if  flag_values_len != flag_meaning_len:
            print('-----')
            print('{}) {}: number of flag_values ({}) different than number of flag_meanings ({}).'.format(e, varname, flag_values_len, flag_meaning_len))
            e += 1
            if verbose:
                print('\tflag_values: {}'.format(ds[varname].attrs['flag_values']))
                print('\tflag_meanings: {}'.format(ds[varname].attrs['flag_meanings']))
            
        invinstring = []
        for char in ds[varname].attrs['flag_meanings']:
            if char in invalid:
                invinstring.append(char)
                
#                 print('------')
#                 print(ds['QC_flag_07_global_shortwave_over_sum'].attrs['flag_meanings'])
#                 assert False
                
        invinstring = set(invinstring)
#         if any(char in invalid for char in ds[varname].attrs['flag_meanings']):
        if len(invinstring) > 0:
            print('-----')
            print('{}) {}: invalid character(s) {} found in flag_meaning'.format(e, varname, ''.join(invinstring)))
            e += 1
            if verbose:
                print('\tflag_meanings: {}'.format(ds[varname].attrs['flag_meanings']))
            
    if 'ancillary_variables' in ds[varname].attrs:
#         raise ValueError()
        for avar in ds[varname].attrs['ancillary_variables'].split():
            if avar not in ds.variables:
                print('-----')
                print('{}) {}: the ancillary_variables {} is not a valid variable ... spell check.'.format(e, varname, avar))
                e += 1
        
        

In [40]:
for varname, value in location_values.items():
    ds[varname].values = value

## save

In [41]:
ds.to_netcdf('../data/Bondville_IL_1995_May_10.nc', 
             format = 'NETCDF4_CLASSIC'
            )

## testing

In [6]:
dst = xr.open_dataset('../data/Bondville_IL_1995_May_10.nc', autoclose=True)

In [18]:
dst.close()

In [169]:
from netCDF4 import Dataset

In [170]:
ds = Dataset('../data/Bondville_IL_1995_May_10.nc')
# ds.close()

In [171]:
ds.variables['time']

<class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: minutes since 1995-05-10 00:00:00
    calendar: proleptic_gregorian
unlimited dimensions: 
current shape = (480,)
filling on, default _FillValue of -2147483647 used

In [151]:
ds.variables['surface_air_pressure']

<class 'netCDF4._netCDF4.Variable'>
float32 surface_air_pressure(time)
    _FillValue: nan
    units: mb
    long_name: Surface air pressure
    standard_name: surface_air_pressure
    comments: Not adjusted to sea level
    _ChunkSize: 480
unlimited dimensions: 
current shape = (480,)
filling on

In [155]:
var = ds.variables['atmospheric_temperature']
var.chunking()

[480]

In [200]:
ds.close()

# parse metadata

In [356]:
def parse_metadata_file(fname = '../data/SURFRAD_QCrad_metadata.txt'):
    def variableblock2attributes(var):
        def clean_variable_attr(variable_line):
            var = variable_line.split('=')[1]
            var = var.strip(' ')
            replacelist = [';', '”', '“', '"']
            for i in replacelist:
                var = var.replace(i, '')#.replace(';').replace('”').replace('“')
            return var
        
        vdic = {}
        vlines = var.split('\n')
        first_line = vlines.pop(0).split()
        if 'global' in first_line:
            vlines.pop(-1)
            vlines.pop(0)
        else:
            vtype = first_line.pop(0)
            vname = first_line.pop(0).replace(';','').replace('(Time=479)', '').replace('(time=479)', '')
            vdic['name'] = vname
            vdic['type'] = vtype
            if len(first_line) != 0:
                vdic['Coordinates'] = first_line.pop(0)

        ancillary_variables = []
        attlist = ['units', 'long_name', 'standard_name', 'comments', 'flag_values', 'flag_meanings', '_FillValue',
                  'title', 'source', 'references', 'institution']
        ignorelist = ['epoch_time', '_ChunkSizes',
                     '_NCProperties', 'Conventions', 'history', 'command_line',
                     '}']
        for vl in vlines:
            found = ignored = False
            for att in attlist:
                if att in vl:
                    catt = clean_variable_attr(vl)
                    if att in vdic.keys():
                        raise ValueError('This attribute ({}) exists already!!'.format(att))
                    vdic[att] = catt
                    found = True
            for att in ignorelist:
                if att in vl:
                    ignored = True

            if found:
                pass
            elif ignored:
                pass
            elif 'ancillary_variables' in vl:
                ancillary_variables.append(clean_variable_attr(vl))
                vdic['ancillary_variables'] = ancillary_variables
            else:
                raise ValueError('No info found in line: {}'.format(vl))

        # some conditioning
        if 'ancillary_variables' in vdic.keys():
            vdic['ancillary_variables'] = ', '.join(vdic['ancillary_variables'])
        return vdic
    
    # open
    full_dict = {}
    with open(fname, 'r') as rein:
        metadatatxtlines = rein.readlines()
    
    # conditioning and splitting into blocks
    metadatatxtlines = [line.replace('NaNf; // float', 'NaN') for line in metadatatxtlines]
    metadatatxtlines = [line.replace('NaNf: // float', 'NaN') for line in metadatatxtlines]
    metadatatxtlines = [line.strip(' ') for line in metadatatxtlines]
    metadatatxt = ''.join(metadatatxtlines)
    metadatablocks = metadatatxt.split('\n\n')

    #get output file name
    full_dict['file_properties'] = {}
    full_dict['file_properties']['fname_out'] = metadatablocks.pop(0).split('\n')[0].split()[1].strip('"')
    pop = metadatablocks.pop(0) # no real information in hear
    
    # get global attributes
    mglobalatts = metadatablocks.pop(-1)
#     return mglobalatts
    # the rest are variables
    mvariables = metadatablocks

    variablesdict = {}
    for e,var in enumerate(mvariables):
        try:
            vdic = variableblock2attributes(var)
    #         if 'flag_values' in vdic.keys():
    #             break
        except ValueError as err:
            print(var)
            print('-----------------')
            print(err)
            break
#         try:
        variablesdict[vdic['name']] = {'attributes': vdic}
#         except:
#             print(vdic)
    full_dict['variables'] = variablesdict
    full_dict['global_variables'] = variableblock2attributes(mglobalatts)
    return full_dict

In [364]:
metadata = parse_metadata_file()

In [365]:
# metadata

In [366]:
columntransrules = {'time': None,
 'solar_zenith_angle':                                                  'Z',
 'distance_from_sun':                                                   'AU',
 'best_estimate_shortwave_irradiance':                                  'BESW',
 'total_downwelling_shortwave_irradiance':                              'GSW',
 'diffuse_downwelling_shortwave_irradiance':                            'DIF',
 'direct_downwelling_shortwave_irradiance':                             'DIR',
 'upwelling_shortwave_irradiance':                                      'SWup',
 'downwelling_longwave_irradiance':                                     'LWdn',
 'upwelling_longwave_irradiance':                                       'LWup',
 'atmospheric_temperature':                                             'Ta',
 'relative_humidity':                                                   'RH',
 'surface_air_pressure':                                                'Prs',
 'downwelling_longwave_case_temperature':                               'LWdTc',
 'downwelling_longwave_dome_temperature':                               'LWdTd',
 'upwelling_longwave_case_temperature':                                 'LWuTc',
 'upwelling_longwave_dome_temperature':                                 'LWuTd',
 'QC_flag_01_thresholds_global_shortwave':                              'qc1',
 'QC_flag_02_thresholds_diffuse_shortwave':                             'qc2',
 'QC_flag_03_thresholds_direct_shortwave':                              'qc3',
 'QC_flag_04_thresholds_upwelling_shortwave':                           'qc4',
 'QC_flag_05_thresholds_downwelling_longwave':                          'qc5',
 'QC_flag_06_thresholds_upwelling_longwave':                            'qc6',
 'QC_flag_07_global_shortwave_over_sum':                                'qc7',
 'QC_flag_08_diffuse_over_global_shortwave':                            'qc8',
 'QC_flag_09_upwelling_shortwave_versus_sum':                           'qc9',
 'QC_flag_10_downwelling_longwave_to_temperature':                      'qc10',
 'QC_flag_11_upwelling_longwave_to_temperature':                        'qc11',
 'QC_flag_12_downwelling_longwave_to_upwelling_longwave':               'qc12',
 'QC_flag_13_downwelling_longwave_case_temperature_vs_air_temperature': 'qc13',
 'QC_flag_14_downwelling_longwave_dome_temperature_vs_air_temperature': 'qc14',
 'QC_flag_15_upwelling_longwave_case_temperature_vs_air_temperature':   'qc15',
 'QC_flag_16_upwelling_longwave_dome_temperature_vs_air_temperature':   'qc16',
 'QC_flag_17_downwelling_longwave_case_temperature_vs_dome_temperature':'qc17',
 'QC_flag_18_upwelling_longwave_case_temperature_vs_dome_temperature':  'qc18',
 'QC_flag_19_air':                                                      'qc19',
 'global_shortwave_irloss_flag':                                        'gflg',
 'diffuse_sortwave_irloss_flag':                                        'dflg',
 'downwelling_shortwave_irradiance_assuming_clear_sky':                 'ClrSW',
 'ir_loss_correction_to_diffuse_shortwave':                             'DifCorr',
 'ir_loss_correction_to_global_shortwave':                              'GSWCorr',
 'downwelling_erythemal_uvb_irradiance':                                'UVB_down',
 'photosynthetically_active_radiation':                                 'PAR_dn',
 'wind_speed':                                                          'WindSp',
 'wind_direction':                                                      'WindDr',
 'latitude': None,
 'longitude': None,
 'altitude': None}

In [369]:
for mvar in  metadata['variables']:
    metadata['variables'][mvar]['raw_data_collmn_name'] = columntransrules[mvar]
    


## Saving metadata for future use

In [374]:
import configobj as cfo

In [390]:
config = cfo.ConfigObj(metadata, indent_type = '    ')

In [391]:
config.filename = '../data/metadata.ini'

In [386]:
for key in metadata.keys():
    config[key] = metadata[key]

In [387]:
config.filename = '../data/metadata.ini'


In [388]:
config.write()

In [389]:
config.

In [379]:
with open('../data/metadata.ini', 'w') as configfile:
    config.write(configfile)

TypeError: write() argument must be str, not bytes

In [376]:
for i in dir(config):
    if 'dict' in i:
        print(i)

__dict__
dict


In [370]:
import configparser as cfp

In [371]:
config = cfp.ConfigParser()

In [372]:
config.read_dict(metadata)

In [373]:
with open('../data/metadata.ini', 'w') as configfile:
    config.write(configfile)

In [317]:
import json

In [318]:
mdjs = json.dumps(metadata, indent=4)
# print(mdjs)

In [319]:
with open('../data/metadata.json', 'w') as raus:
    raus.write(mdjs)

# Read data

In [320]:
fname = '../data/bon_19950510.qdat'
data = pd.read_csv(fname,delim_whitespace=True)

# creating timestamp
datetimestr = data.Date.apply(lambda x: '{0:0>8}'.format(x)) + data.Time.apply(lambda x: '{0:0>4}'.format(x))
data.index = pd.to_datetime(datetimestr, format = "%Y%m%d%H%M") 
data.index.name = 'Time'

# removing obsolete columns
data = data.drop(['Date', 'Time'], axis=1)

In [321]:
data

,BESW,GSW,DIF,DIR,SWup,LWdn,LWup,Ta,RH,Prs,...,dflg,Z,AU,ClrSW,DifCorr,GSWCorr,UVB_down,PAR_dn,WindSp,WindDr
Time,,,,,,,,,,,,,,,,,,,,,
1995-05-10 00:00:00,17.9,17.9,18.0,-0.1,1.7,387.4,420.8,295.0,66.1,-9999.9,...,0,80.7,1.00964,120.4,0.0,5.8,1.0,7.9,8.3,173.2
1995-05-10 00:03:00,15.3,15.3,15.5,-0.2,1.8,387.8,420.2,294.9,66.4,-9999.9,...,0,81.3,1.00965,111.9,0.0,5.8,0.9,6.7,7.8,172.7
1995-05-10 00:06:00,12.4,12.4,12.5,-0.2,1.7,388.4,419.7,295.1,66.4,-9999.9,...,0,81.9,1.00965,103.4,0.0,5.4,0.7,5.5,8.0,172.3
1995-05-10 00:09:00,10.7,10.7,10.9,-0.2,1.6,388.6,419.7,294.8,66.6,-9999.9,...,0,82.4,1.00965,95.2,0.0,5.4,0.6,4.6,7.8,172.9
1995-05-10 00:12:00,9.2,9.2,9.4,-0.1,1.2,389.8,419.4,295.1,66.4,-9999.9,...,0,83.0,1.00965,87.0,0.0,4.9,0.5,4.0,8.0,171.3
1995-05-10 00:15:00,8.9,8.9,9.1,-0.1,1.1,389.9,419.3,295.0,66.2,-9999.9,...,0,83.5,1.00965,79.0,0.0,4.9,0.4,3.8,7.9,169.6
1995-05-10 00:18:00,8.1,8.1,8.3,-0.1,1.0,390.2,419.3,294.9,66.2,-9999.9,...,0,84.0,1.00965,71.1,0.0,4.5,0.4,3.6,7.9,166.7
1995-05-10 00:21:00,8.7,8.7,8.9,-0.1,1.4,390.6,418.7,294.8,66.6,-9999.9,...,0,84.6,1.00965,63.4,0.0,4.5,0.4,3.7,7.7,167.3
1995-05-10 00:24:00,6.8,6.8,6.9,-0.1,1.2,392.2,418.6,294.7,67.0,-9999.9,...,0,85.1,1.00965,55.8,0.0,4.1,0.4,3.1,8.1,167.8


# Create dataset

In [330]:
ds = xr.Dataset()

for var in metadata['variables']:
    print(var)
    if var == 'time':
        vartime = metadata['variables'][var]
        continue
    vardic = metadata['variables'][var]
    dat_col_name = vardic['raw_data_collmn_name']
    if dat_col_name:
        ds[var] = data.loc[:,dat_col_name]
    else:
        if var == 'latitude':
            ds[var] = 40.05192
        elif var == 'longitude':
            ds[var] = 271.62692
        elif var == 'altitude':
            ds[var] = 230.
    for atr in vardic['attributes']:
        if atr == 'name':
            continue
        if vardic['attributes'][atr] == 'NaN':
            ds[var].attrs[atr] = np.nan
        else:
            ds[var].attrs[atr] = vardic['attributes'][atr]   
#     ds.to_netcdf('../data/' + metadata['fname_out'])

for gv in metadata['global_variables']:
    ds.attrs[gv] = metadata['global_variables'][gv]

for atr in vartime['attributes']:
    if atr in ['name', 'units', 'type']:
            continue
    ds.Time.attrs[atr] = vartime['attributes'][atr]

time
solar_zenith_angle
distance_from_sun
best_estimate_shortwave_irradiance
total_downwelling_shortwave_irradiance
diffuse_downwelling_shortwave_irradiance
direct_downwelling_shortwave_irradiance
upwelling_shortwave_irradiance
downwelling_longwave_irradiance
upwelling_longwave_irradiance
atmospheric_temperature
relative_humidity
surface_air_pressure
downwelling_longwave_case_temperature
downwelling_longwave_dome_temperature
upwelling_longwave_case_temperature
upwelling_longwave_dome_temperature
QC_flag_01_thresholds_global_shortwave
QC_flag_02_thresholds_diffuse_shortwave
QC_flag_03_thresholds_direct_shortwave
QC_flag_04_thresholds_upwelling_shortwave
QC_flag_05_thresholds_downwelling_longwave
QC_flag_06_thresholds_upwelling_longwave
QC_flag_07_global_shortwave_over_sum
QC_flag_08_diffuse_over_global_shortwave
QC_flag_09_upwelling_shortwave_versus_sum
QC_flag_10_downwelling_longwave_to_temperature
QC_flag_11_upwelling_longwave_to_temperature
QC_flag_12_downwelling_longwave_to_upwellin

In [331]:
ds

<xarray.Dataset>
Dimensions:                                                               (Time: 480)
Coordinates:
  * Time                                                                  (Time) datetime64[ns] 1995-05-10 ...
Data variables:
    solar_zenith_angle                                                    (Time) float64 80.7 ...
    distance_from_sun                                                     (Time) float64 1.01 ...
    best_estimate_shortwave_irradiance                                    (Time) float64 17.9 ...
    total_downwelling_shortwave_irradiance                                (Time) float64 17.9 ...
    diffuse_downwelling_shortwave_irradiance                              (Time) float64 18.0 ...
    direct_downwelling_shortwave_irradiance                               (Time) float64 -0.1 ...
    upwelling_shortwave_irradiance                                        (Time) float64 1.7 ...
    downwelling_longwave_irradiance                                     

In [334]:
ds.to_netcdf('../data/' + metadata['fname_out'])

# testing

In [655]:
from netCDF4 import Dataset

In [656]:
ds = Dataset('../data/Bondville_IL_1995_May_10.nc')
# ds.close()

In [660]:
var = ds.variables['solar_zenith_angle']

In [661]:
var.chunking()

'contiguous'

In [662]:
ds.close()

In [344]:
ds.variables['downwelling_longwave_dome_temperature']

<class 'netCDF4._netCDF4.Variable'>
float64 downwelling_longwave_dome_temperature(Time)
    _FillValue: nan
    type: float
    units: K
    long_name: Downwelling longwave pyrgeometer dome temperature
unlimited dimensions: 
current shape = (480,)
filling on

In [ ]:
<class 'netCDF4._netCDF4.Variable'>
float64 downwelling_longwave_dome_temperature(time)
    _FillValue: nan
unlimited dimensions: 
current shape = (479,)
filling on

# structure of nc-files in old script collection

In [156]:
fname = '../scripts/Radiation NetCDF Scripts/NetCDF_output/Bondville_IL_1995_01_01.nc'

In [157]:
dst = xr.open_dataset(fname, decode_times=False)

<xarray.DataArray 'Time' (Time: 479)>
array([   3,    6,    9, ..., 1431, 1434, 1437], dtype=int32)
Coordinates:
  * Time     (Time) int32 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 48 51 54 ...
Attributes:
    long_name:  Time (beginning at midnight)
    units:      minutes since 1995-01-01 00:00:00
    calendar:   none

In [66]:
dst.close()

In [514]:
dsold = Dataset(fname)

In [520]:
dsold

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6
    title: QCRad V3 Dataset from NOAA/ESRL/GMD/GRAD Surfrad Radiation Archive
    date: Data recorded on 01/01/1995
    location: Bondville_IL
    latitude: 40.05192 N
    longitude: 271.62691 E
    elevation: 230 meters
    source: NOAA/ESRL/GMD/GRAD Surfrad Radiation Archive
    references: Online: https://www.esrl.noaa.gov/gmd/grad/surfrad/index.html
    history: [2018-01-08 14:57] Created compressed netCDF4 dataset
    institution: National Oceanic and Atmospheric Administration (NOAA) - David Skaggs Research Center - Boulder, CO
    comment: Surface radiation data saved in daily files by test site
    dimensions(sizes): Time(479)
    variables(dimensions): int32 Time(Time), int32 year(Time), int32 month(Time), int32 day(Time), float32 latitude(Time), float32 longitude(Time), int32 altitude(Time), float32 best_estimate_shortwave(Time), float32 total_shortwave(Time), float32

In [516]:
for k in dsold.variables.keys():
    print(k)

Time
year
month
day
latitude
longitude
altitude
best_estimate_shortwave
total_shortwave
diffuse_downwelling_shortwave_flux_in_air
direct_downwelling_shortwave_flux_in_air
upwelling_shortwave_flux_in_air
downwelling_longwave_flux_in_air
upwelling_longwave_flux_in_air
air_temperature
relative_humidity
surface_air_pressure
downwelling_longwave_case_temperature
downwelling_longwave_dome_temperature
upwelling_longwave_case_temperature
upwelling_longwave_dome_temperature
qc01
qc02
qc03
qc04
qc05
qc06
qc07
qc08
qc09
qc10
qc11
qc12
qc13
qc14
qc15
qc16
qc17
qc18
qc19
g_flag
d_flag
solar_zenith_angle
distance_from_sun
downwelling_shortwave_flux_in_air_assuming_clear_sky
ir_loss_correction_to_diffuse_shortwave
ir_loss_correction_to_global_shortwave
downwelling_erythemal_uvb_flux_in_air
surface_downwelling_photosynthetic_radiative_flux_in_air
wind_speed
wind_from_direction


In [152]:
fname = '../scripts/Radiation NetCDF Scripts/NetCDF_output/Bondville_IL_1995_01_01.nc'
dsold = xr.open_dataset(fname, decode_times=False)

In [ ]:
40.05192, 271.62692, 230

In [153]:
dsold

<xarray.Dataset>
Dimensions:                                                   (Time: 479)
Coordinates:
  * Time                                                      (Time) int32 3 ...
Data variables:
    year                                                      (Time) int32 ...
    month                                                     (Time) int32 ...
    day                                                       (Time) int32 ...
    latitude                                                  (Time) float32 ...
    longitude                                                 (Time) float32 ...
    altitude                                                  (Time) int32 ...
    best_estimate_shortwave                                   (Time) float32 ...
    total_shortwave                                           (Time) float32 ...
    diffuse_downwelling_shortwave_flux_in_air                 (Time) float32 ...
    direct_downwelling_shortwave_flux_in_air                  (Time) float32 .

In [91]:
for k in ds.variables.keys():
    print(k)

Time
year
month
day
latitude
longitude
altitude
best_estimate_shortwave
total_shortwave
diffuse_downwelling_shortwave_flux_in_air
direct_downwelling_shortwave_flux_in_air
upwelling_shortwave_flux_in_air
downwelling_longwave_flux_in_air
upwelling_longwave_flux_in_air
air_temperature
relative_humidity
surface_air_pressure
downwelling_longwave_case_temperature
downwelling_longwave_dome_temperature
upwelling_longwave_case_temperature
upwelling_longwave_dome_temperature
qc01
qc02
qc03
qc04
qc05
qc06
qc07
qc08
qc09
qc10
qc11
qc12
qc13
qc14
qc15
qc16
qc17
qc18
qc19
g_flag
d_flag
solar_zenith_angle
distance_from_sun
downwelling_shortwave_flux_in_air_assuming_clear_sky
ir_loss_correction_to_diffuse_shortwave
ir_loss_correction_to_global_shortwave
downwelling_erythemal_uvb_flux_in_air
surface_downwelling_photosynthetic_radiative_flux_in_air
wind_speed
wind_from_direction


# structure of an arbitrary file

In [162]:
fname = '/Volumes/HTelg_4TB_Backup/arm_data/SGP/1twr10xC1/sgp1twr10xC1.b1.20110101.000000.cdf'

In [163]:
dst = xr.open_dataset(fname)

In [164]:
dst.time

<xarray.DataArray 'time' (time: 1440)>
array(['2011-01-01T00:00:00.000000000', '2011-01-01T00:01:00.000000000',
       '2011-01-01T00:02:00.000000000', ..., '2011-01-01T23:57:00.000000000',
       '2011-01-01T23:58:00.000000000', '2011-01-01T23:59:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2011-01-01 2011-01-01T00:01:00 ...
Attributes:
    long_name:  Time offset from midnight